In [1]:
import torch
from torch_geometric.data import InMemoryDataset, download_url
from torch_geometric.data import DataLoader

import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter   

from tqdm import tqdm
import random
import numpy as np

In [2]:
class train_5000(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(train_5000, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['data.pt']



    def process(self):
        # Read data into huge `Data` list.



        data, slices = self.collate(dl)
        torch.save((data, slices), self.processed_paths[0])

In [3]:
import torch
from torch_geometric.data import InMemoryDataset, download_url


class val_5000(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(val_5000, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['data.pt']



    def process(self):
        # Read data into huge `Data` list.



        data, slices = self.collate(dl_test)
        torch.save((data, slices), self.processed_paths[0])

In [4]:

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')
        self.lin = torch.nn.Linear(in_channels, out_channels)


    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        x = self.lin(x)
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype) + 1
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x, norm=norm)

    def message(self, x_j, norm):

        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):

        return aggr_out

class dr_bc(nn.Module):
    def __init__(self,):
        super(dr_bc, self).__init__()

        self.fc1 = nn.Linear(3, 128)
        self.relu = nn.LeakyReLU()
        
        self.gru1 = nn.GRU(128, 128)
        self.gru2 = nn.GRU(128, 128)
        self.gru3 = nn.GRU(128, 128)
        self.gru4 = nn.GRU(128, 128)
        
        
        self.gcn1 = GCNConv(128, 128)
        self.gcn2 = GCNConv(128, 128)
        self.gcn3 = GCNConv(128, 128)
        self.gcn4 = GCNConv(128, 128)
        self.gcn5 = GCNConv(128, 128)

        # Decoder
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.LeakyReLU()
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x, edge_index):
        
        x = self.fc1(x)
        x_1v = self.relu(x)
        x_1v = x_1v/torch.norm(x_1v, p=2)
        x_2n = self.gcn1(x_1v, edge_index)
        x_2v, _ = self.gru1(x_2n.view(1, *x_2n.shape), x_1v.view(1, *x_1v.shape))
        x_3n = self.gcn2(x_2v[0], edge_index)
        x_3v, _ = self.gru2(x_3n.view(1, *x_3n.shape), x_2v)
        x_4n = self.gcn3(x_3v[0], edge_index)
        x_4v, _ = self.gru3(x_4n.view(1, *x_4n.shape), x_3v)
        x_5n = self.gcn4(x_4v[0], edge_index)
        x_5v, _ = self.gru4(x_5n.view(1, *x_5n.shape), x_4v)

        
        # max
        l = torch.stack([x_1v, x_2v[0], x_3v[0], x_4v[0], x_5v[0]])
        emb = torch.max(l, dim=0).values


        # decoder
        y = self.fc2(emb)
        y = self.relu2(y)
        y = self.fc3(y)
        return y



In [5]:

# class GCNConv(MessagePassing):
#     def __init__(self, in_channels, out_channels):
#         super(GCNConv, self).__init__(aggr='add')
#         self.lin = torch.nn.Linear(in_channels, out_channels)


#     def forward(self, x, edge_index):
#         edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
#         x = self.lin(x)
#         row, col = edge_index
#         deg = degree(row, x.size(0), dtype=x.dtype) + 1
#         deg_inv_sqrt = deg.pow(-0.5)
#         norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

#         return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x, norm=norm)

#     def message(self, x_j, norm):

#         return norm.view(-1, 1) * x_j

#     def update(self, aggr_out):

#         return aggr_out

# class dr_bc(nn.Module):
#     def __init__(self,):
#         super(dr_bc, self).__init__()

#         self.fc1 = nn.Linear(3, 128)
#         self.relu = nn.LeakyReLU()
        
#         self.gru = nn.GRU(128, 128)
        
        
#         self.gcn1 = GCNConv(128, 128)
#         self.gcn2 = GCNConv(128, 128)
#         self.gcn3 = GCNConv(128, 128)
#         self.gcn4 = GCNConv(128, 128)
#         self.gcn5 = GCNConv(128, 128)

#         # Decoder
#         self.fc2 = nn.Linear(128, 64)
#         self.relu2 = nn.LeakyReLU()
#         self.fc3 = nn.Linear(64, 1)

#     def forward(self, x, edge_index):
        
#         x = self.fc1(x)
#         x_1v = self.relu(x)
#         x_1v = x_1v/torch.norm(x_1v, p=2)
#         x_2n = self.gcn1(x_1v, edge_index)
#         x_2v, _ = self.gru(x_2n.view(1, *x_2n.shape), x_1v.view(1, *x_1v.shape))
#         x_3n = self.gcn2(x_2v[0], edge_index)
#         x_3v, _ = self.gru(x_3n.view(1, *x_3n.shape), x_2v)
#         x_4n = self.gcn3(x_3v[0], edge_index)
#         x_4v, _ = self.gru(x_4n.view(1, *x_4n.shape), x_3v)
#         x_5n = self.gcn4(x_4v[0], edge_index)
#         x_5v, _ = self.gru(x_5n.view(1, *x_5n.shape), x_4v)

        
#         # max
#         l = torch.stack([x_1v, x_2v[0], x_3v[0], x_4v[0], x_5v[0]])
#         emb = torch.max(l, dim=0).values


#         # decoder
#         y = self.fc2(emb)
#         y = self.relu2(y)
#         y = self.fc3(y)
#         return y



In [6]:
batch_size = 25
train_d = train_5000(root='C:/Users/bbb50/Desktop/Python/hw1/train_5000')
train_loader = DataLoader(train_d, batch_size=batch_size, shuffle=True)

val_d = val_5000(root='C:/Users/bbb50/Desktop/Python/hw1/val_5000')
val_loader = DataLoader(val_d, batch_size=batch_size, shuffle=True)

In [7]:
# node_num = 5000

# np_list = []
# for n in tqdm(range(50)):
#     node_pair = []
#     for i in range(batch_size):
#         for j in range(node_num*5):
#             node_pair.append(random.sample(range(i*node_num, node_num+i*node_num), 2))
#     np_list.append(node_pair)
# np_list = torch.tensor( np_list, dtype=torch.long)
# torch.save(np_list, 'node_pair_5000_size25')

In [8]:
node_pair = torch.load('node_pair_5000_size25').cuda()

In [9]:
writer = SummaryWriter('./path/to/log')

In [10]:
def train(data, node_pair):
    
    model.train()
    optimizer.zero_grad() 

    output = model(data.x.cuda(), data.edge_index.cuda()) 

    if data.num_graphs == batch_size:
        pair = node_pair
    else:
        pair = node_pair[:data.num_graphs*node_num*5]

    yij = output[pair[:, 0]] - output[pair[:, 1]]
    bij = torch.sigmoid((data.y.cuda()[pair[:, 0]] - data.y.cuda()[pair[:, 1]]))
    bij = bij.view(yij.shape)

    train_loss = criterion(yij, bij)
    train_loss.backward()
    optimizer.step()
    
    return train_loss

def val(data, node_pair):
    with torch.no_grad():
        
        model.eval()
        output = model(data.x.cuda(), data.edge_index.cuda()) 

        if data.num_graphs == batch_size:
            pair = node_pair
        else:
            pair = node_pair[:data.num_graphs*node_num*5]

        yij = output[pair[:, 0]] - output[pair[:, 1]]
        bij = torch.sigmoid((data.y.cuda()[pair[:, 0]] - data.y.cuda()[pair[:, 1]]))
        bij = bij.view(yij.shape)

        val_loss = criterion(yij, bij)
    
    return val_loss

In [13]:
model = dr_bc()

criterion = nn.BCEWithLogitsLoss()
checkpoint = torch.load('5000_train.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

epoch_start = checkpoint['epoch']
epochs = 10000
node_num = 5000
count = 0

early_count = 0

In [14]:
epoch_start

23

In [14]:
# model = dr_bc()
# criterion = nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# model = model.cuda()
# epochs = 10000
# node_num = 5000
# count = 0
# early_count = 0

In [15]:

for epoch in range(epoch_start, epochs):
    
    n_p = node_pair[0]
    train_loss = []
    val_loss = []
    
    for train_data in tqdm(train_loader):
        t_loss = train(train_data, n_p)
        train_loss.append(t_loss.item())

    
    for val_data in val_loader:
        v_loss = val(val_data, node_pair[0])
        val_loss.append(v_loss.item()) 

    new_train_loss = np.array(train_loss).mean()
    new_val_loss = np.array(val_loss).mean()
    
    writer.add_scalar('loss/train_loss', new_train_loss, epoch)
    writer.add_scalar('loss/val_loss', new_val_loss, epoch)
    
    if epoch > epoch_start:
        if new_val_loss > last_val_loss:
            count = count + 1
        elif new_val_loss < last_val_loss:

            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': new_train_loss,
            }, "5000_train.pth")
            last_val_loss = new_val_loss
            count = 0
            
    if epoch == epoch_start:
        last_val_loss = new_val_loss
    if count> 50:
        n_p = node_pair[np.random.randint(0, node_pair.shape[0], 1)[0]]
        early_count = early_count + 1
        
    if early_count > 15:
        break

    log = 'Epoch: {:05d}, train_loss: {:}, val_loss: {:}, last_val_loss: {:}, not_improve_count: {:02d}'
    print(log.format(epoch, new_train_loss, new_val_loss, last_val_loss, count))

  0%|                                                                                          | 0/280 [00:00<?, ?it/s]

Epoch: 00023, train_loss: 0.6931467249989509, val_loss: 0.6931466857592264, last_val_loss: 0.6931466857592264, not_improve_count: 00


  0%|                                                                                          | 0/280 [00:00<?, ?it/s]

Epoch: 00024, train_loss: 0.6931467228702136, val_loss: 0.6931467254956564, last_val_loss: 0.6931466857592264, not_improve_count: 01


 14%|██████████▉                                                                      | 38/280 [01:00<06:23,  1.58s/it]


KeyboardInterrupt: 

In [13]:
# # train_loss_l = []
# # val_loss_l = []
# for epoch in range(epochs):
    
#     train_loss = []
#     val_loss = []
    
#     for train_data in tqdm(train_loader):
#         t_loss = train(train_data, node_pair)
#         train_loss.append(t_loss.item())
# #         print(t_loss.item())
    
# #     print('/')
# #     print('validation')
    
#     for val_data in val_loader:
#         v_loss = val(val_data, node_pair)
#         val_loss.append(v_loss.item()) 
# #         print(v_loss.item())
    
# #     train_loss_l.append(train_loss)
# #     val_loss_l.append(val_loss)
#     new_train_loss = np.array(train_loss).mean()
#     new_val_loss = np.array(val_loss).mean()
    
#     writer.add_scalar('loss/train_loss', new_train_loss, epoch)
#     writer.add_scalar('loss/val_loss', new_val_loss, epoch)
    
#     if epoch > 1:
#         if new_val_loss > last_val_loss:
#             count = count + 1
#         else:
#             torch.save(model.state_dict(), "weight.pth")
#             count = 0
#     if count> 50:
#         break
 
#     last_val_loss = new_val_loss

#     log = 'Epoch: {:05d}, train_loss: {:}, val_loss: {:}, not_improve_count: {:02d}'
#     print(log.format(epoch, new_train_loss, new_val_loss, count))

  0%|                                                                                          | 0/280 [00:00<?, ?it/s]

Epoch: 00000, train_loss: 0.6931468078068325, val_loss: 0.6931467851003011, not_improve_count: 00


  0%|                                                                                          | 0/280 [00:00<?, ?it/s]

Epoch: 00001, train_loss: 0.6931467824748584, val_loss: 0.6931467652320862, not_improve_count: 00


  0%|                                                                                          | 0/280 [00:00<?, ?it/s]

...

Epoch: 00007, train_loss: 0.6931467584201267, val_loss: 0.6931467453638712, not_improve_count: 00


  0%|                                                                                          | 0/280 [00:00<?, ?it/s]

Epoch: 00008, train_loss: 0.6931467511824199, val_loss: 0.6931467453638712, not_improve_count: 00


100%|████████████████████████████████████████████████████████████████████████████████| 280/280 [01:09<00:00,  4.02it/s]


Epoch: 00009, train_loss: 0.6931467539497784, val_loss: 0.6931467652320862, not_improve_count: 01


In [16]:
# print(log.format(epoch, new_train_loss, new_val_loss))

Epoch: 00000, train_loss: 0.6977, val_loss: 0.6931


In [24]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()